# GLM data prep

Create a table of data for each recording.
Each row is a millisecond (data only from bouts).
Variables include speeds + shifted speeds, curvature of the track, firing rate...


In [1]:
# imports
import sys
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import interpolate
from fcutils.progress import track
from fcutils.maths import derivative
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

sys.path.append("./")
sys.path.append(r"C:\Users\Federico\Documents\GitHub\pysical_locomotion")


from analysis.ephys.utils import get_recording_names, get_data, get_session_bouts

save_folder = Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\ephys")

cache = Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\ephys\GLM\data")
recordings = get_recording_names()


[06/20/22 09:29:04] DEBUG    DPL - Saving data and logs to logs                   ]8;id=849616;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py\core.py]8;;\:]8;id=513604;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py#118\118]8;;\

                    DEBUG    Saving log file to: logs\log.log                     ]8;id=421768;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py\core.py]8;;\:]8;id=4652;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py#119\119]8;;\

Connecting root@127.0.0.1:3306


[06/20/22 09:29:05] DEBUG    Connected to database: LOCOMOTION4                ]8;id=882393;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py\__init__.py]8;;\:]8;id=115898;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py#11\11]8;;\

## Parameters

In [2]:
curvature_horizon = 30
curvature_sampling_spacing = 5
curv_sample_points = np.arange(0, curvature_horizon+curvature_sampling_spacing, curvature_sampling_spacing)

track_downsample_factor = 25

firing_rate_gaussian = 250 # width in ms

### Track curvature
Sample the track curvature for N future positions given each track position

In [3]:
track_data = pd.read_json(r"C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\ephys\track.json").iloc[::track_downsample_factor]
track_data = track_data.reset_index(drop=True)
S_f = track_data.S.values[-1]
track_data

,S,X,Y,curvature,width,θ
0,0.000000,20.005419,32.926672,1.397851e-08,3.600000,-1.562696
1,0.034248,20.005696,32.892425,2.407125e-04,3.596612,-1.562692
2,0.068464,20.005974,32.858210,4.834723e-04,3.590541,-1.562680
3,0.102616,20.006251,32.824059,7.303402e-04,3.584550,-1.562659
4,0.136672,20.006529,32.790004,9.834296e-04,3.578645,-1.562630
...,...,...,...,...,...,...
3996,265.273224,3.999664,7.342840,3.530815e-05,3.600000,-1.570627
3997,265.334935,3.999675,7.281129,2.648609e-05,3.600000,-1.570625
3998,265.396642,3.999685,7.219422,1.766084e-05,3.600000,-1.570624
3999,265.458347,3.999696,7.157717,8.831522e-06,3.600000,-1.570623


In [4]:
# load track from json
k_shifts = np.arange(curvature_horizon+1)
curv_shifted = {
    **{f"k_{k}":[] for k in k_shifts},
    **{f"idx_{k}":[] for k in k_shifts},
}
for i, s in enumerate(track_data.S):
    for k in k_shifts:
        if s + k < S_f:
            select = track_data.loc[track_data.S >= s + k]
            curv_shifted[f"idx_{k}"].append(select.index[0])
            curv_shifted[f"k_{k}"].append(select["curvature"].iloc[0])
        else:
            curv_shifted[f"k_{k}"].append(np.nan)
            curv_shifted[f"idx_{k}"].append(np.nan)

    # break

for k,v in curv_shifted.items():
    track_data.insert(2, k, v)
track_data.head()

,S,X,idx_60,idx_59,idx_58,idx_57,idx_56,idx_55,idx_54,idx_53,...,k_5,k_4,k_3,k_2,k_1,k_0,Y,curvature,width,θ
0,0.000000,20.005419,899.0,884.0,868.0,852.0,837.0,821.0,805.0,790.0,...,0.014435,0.011079,0.011057,0.004578,0.005121,1.397851e-08,32.926672,1.397851e-08,3.600000,-1.562696
1,0.034248,20.005696,900.0,884.0,868.0,853.0,837.0,822.0,806.0,790.0,...,0.014729,0.011079,0.011057,0.004907,0.004178,2.407125e-04,32.892425,2.407125e-04,3.596612,-1.562692
2,0.068464,20.005974,900.0,885.0,869.0,853.0,838.0,822.0,806.0,791.0,...,0.015033,0.011253,0.011257,0.004907,0.003339,4.834723e-04,32.858210,4.834723e-04,3.590541,-1.562680
3,0.102616,20.006251,901.0,885.0,870.0,854.0,838.0,823.0,807.0,791.0,...,0.015033,0.011253,0.011431,0.005266,0.002610,7.303402e-04,32.824059,7.303402e-04,3.584550,-1.562659
4,0.136672,20.006529,901.0,886.0,870.0,854.0,839.0,823.0,808.0,792.0,...,0.015348,0.011434,0.011431,0.005266,0.001990,9.834296e-04,32.790004,9.834296e-04,3.578645,-1.562630


## Process data

In [5]:

def upsample_frames_to_ms(var):
    """
        Interpolates the values of a variable expressed in frams (60 fps)
        to values expressed in milliseconds.
    """
    t_60fps = np.arange(len(var)) / 60
    f = interpolate.interp1d(t_60fps, var)

    # t_1000fps = np.arange(0, t_60fps[-1], step=1/1000)
    t_200fps = np.arange(0, t_60fps[-1], step=1/200)
    interpolated_variable_values = f(t_200fps)
    return interpolated_variable_values


In [6]:

def gaussian(x, s):
    return (
        1.0
        / np.sqrt(2.0 * np.pi * s ** 2)
        * np.exp(-(x ** 2) / (2.0 * s ** 2))
    )


def calc_firing_rate(spikes_train: np.ndarray, dt: int = 10):
    """
        Computes the firing rate given a spikes train (wether there is a spike or not at each ms).
        Using a gaussian kernel with standard deviation = dt/2 [dt is in ms]
    """
    # create kernel & get area under the curve
    k = np.array([gaussian(x, dt / 2) for x in np.linspace(-2*dt, 2*dt, dt)])
    auc = np.trapz(k)

    # get firing rate
    frate = (
        np.convolve(spikes_train, k, mode="same") / auc * 1000
    )  # times 1000 to go from ms to seconds

    
    return frate[::5]  # sample every 5 ms -> 200 fps
    # return frate


def make_shuffled_units(units):
    """
        For each unit make shuffled copies in which 
        the firing rate is offset by some ammount looping
        around the start/end of the session
    """
    N = 100
    shuffled_units = dict(unit_id=[], firing_rate_ms=[])
    for i, unit in units.iterrows():
        for n in range(N):
            shuffle = np.random.randint(10 * 200, 100 * 200)  # shuffle between 10 and 30 seconds
            shuffled_units["unit_id"].append(f"{unit.unit_id}_shuffle_{n}")

            frate = unit.firing_rate_ms
            frate = np.hstack([frate[shuffle:], frate[:shuffle]])
            shuffled_units["firing_rate_ms"].append(frate)

    # merge units and shuffle units in a single dataframe
    shuffled_units = pd.DataFrame(shuffled_units)
    units = pd.concat([units, shuffled_units], ignore_index=True)
    return units

#### main data loader

In [7]:
# upsample
def load_get_recording_data(REC): 
    # load data
    units, left_fl, right_fl, left_hl, right_hl, body = get_data(REC)

    out_bouts = get_session_bouts(REC, complete=None)
    in_bouts = get_session_bouts(REC, direction="inbound", complete=None)

    v = upsample_frames_to_ms(body.speed)
    omega = upsample_frames_to_ms(body.thetadot)

    ms_per_frame = 1000 / 200
    _250ms = int(250 / ms_per_frame)
    _500ms = int(500 / ms_per_frame)
    _1000ms = int(1000 / ms_per_frame)

    v_250ms = np.hstack([v[_250ms:], v[_250ms] * np.ones(_250ms)]) - v
    v_500ms = np.hstack([v[_500ms:], v[_500ms] * np.ones(_500ms)]) - v
    v_1000ms = np.hstack([v[_1000ms:], v[_1000ms] * np.ones(_1000ms)]) - v

    omega_500ms = np.hstack([omega[_500ms:], omega[_500ms] * np.ones(_500ms)]) - omega
    omega_250ms = np.hstack([omega[_250ms:], omega[_250ms] * np.ones(_250ms)]) - omega
    omega_1000ms = np.hstack([omega[_1000ms:], omega[_1000ms] * np.ones(_1000ms)]) - omega


    # get unit firing rate in milliseconds
    units = units.loc[units.brain_region.isin(["MOs", "MOs1", "MOs2/3", "MOs5", "MOs6a", "MOs6b"])]
    frates = []
    for i, unit in units.iterrows():
        time = np.zeros(len(v) * 5)  # time in milliseconds
        spikes_times = np.int64(np.round(unit.spikes_ms))
        spikes_times = spikes_times[spikes_times < len(time)]
        time[spikes_times] = 1
        frates.append(calc_firing_rate(time, dt=firing_rate_gaussian))  # firing rate at 200fps
    units["firing_rate_ms"] = frates
    units = units[["unit_id", "firing_rate_ms"]]  # discard unnecessary columns

    # add shuffled units
    units = make_shuffled_units(units)

    return units, body, pd.concat([out_bouts, in_bouts]).reset_index(), v, omega, v_250ms, omega_250ms, v_500ms, omega_500ms, v_1000ms, omega_1000ms

### Collect data for all bouts

In [25]:
for REC in recordings:
    savepath = cache / f"{REC}_bouts.h5"
    if savepath.exists():
        print(f"{REC}_bouts.h5 already exists")
        continue
    
    print(f"Doing   {REC}")
    out_bouts = get_session_bouts(REC, complete=None)
    in_bouts = get_session_bouts(REC, direction="inbound", complete=None)
    allbouts = pd.concat([out_bouts, in_bouts]).reset_index()

    bouts_files = list(cache.glob(f"{REC}_bout_*.h5"))
    if len(bouts_files) < len(allbouts):
        print(f"    Not all bouts were saved for {REC}")
        continue

    bouts_data = []
    for i, bout in allbouts.iterrows():
        f = cache / f"{REC}_bout_{bout.start_frame}.h5"
        if not f.exists():
            break
        try:
            bouts_data.append(pd.read_hdf(f, key="data"))
        except AttributeError:
            print(f"    {f} was too hard to read")
            continue
    bouts_data = pd.concat(bouts_data)

    # print(f" Got all data ({bouts_data.shape}), removing outliers")
    # del out_bouts; del in_bouts; del allbouts
    # bouts_data[(np.abs(stats.zscore(bouts_data)) < 3).all(axis=1)]


    print(" Saving data")
    bouts_data.to_hdf(savepath, key="data")
    print(" Saved all data")


FC_220408_BAA1101192_hairpin_bouts.h5 already exists
FC_220409_BAA1101192_hairpin_bouts.h5 already exists
FC_220410_BAA1101192_hairpin_bouts.h5 already exists
FC_220411_BAA1101192_hairpin_bouts.h5 already exists
FC_220412_BAA1101192_hairpin_bouts.h5 already exists
FC_220413_BAA1101192_hairpin_bouts.h5 already exists
FC_220414_BAA1101192_hairpin_bouts.h5 already exists
FC_220415_BAA1101192_hairpin_bouts.h5 already exists
FC_220432_BAA1101192_hairpin_bouts.h5 already exists
FC_220433_BAA1101192_hairpin_bouts.h5 already exists
FC_220434_BAA1101192_hairpin_bouts.h5 already exists
Doing   FC_220435_BAA1101192_hairpin
 Saving data
 Saved all data


In [ ]:
# first, for each recording and each bout save a .h5 file. 
# Then load it back and merge them toghether. This is to avoid memory issues.

for REC in recordings:
    if (cache / f"{REC}_bouts.h5").exists():
        print(f"{REC}_bouts.h5 already exists")
        continue
    print(f"Processing {REC}")

    units, body, bouts, v, omega, v_250ms, omega_250ms, v_500ms, omega_500ms, v_1000ms, omega_1000ms = load_get_recording_data(REC)
    print("     got all data")

    for i, bout in track(bouts.iterrows(), total=len(bouts), description=REC):
        bout_savepath = cache / f"{REC}_bout_{bout.start_frame}.h5"
        if bout_savepath.exists():
            print(f"{REC}_bout_{bout.start_frame}.h5 already exists")
            continue

        data = {
            **dict(
                s=[],
                sdot=[],
                v=[],
                dv_250ms=[],
                dv_500ms=[],
                dv_1000ms=[],
                omega=[],
                domega_250ms=[],
                domega_500ms=[],
                domega_1000ms=[],
            ),
            **{f"curv_{k}cm":[] for k in curv_sample_points},
            **{unit:[] for unit in units.unit_id.values},
        }


        S = upsample_frames_to_ms(bout.s)
        data['s'].extend(S)
        data['sdot'].extend(derivative(S) * 60)

        start_ms = int(bout.start_frame / 60 * 200)
        end_ms = start_ms + len(S)
        data['v'].extend(v[start_ms : end_ms])
        data['dv_250ms'].extend(v_250ms[start_ms : end_ms])
        data['dv_500ms'].extend(v_500ms[start_ms : end_ms])
        data['dv_1000ms'].extend(v_1000ms[start_ms : end_ms])
        data['omega'].extend(omega[start_ms : end_ms])
        data['domega_250ms'].extend(omega_250ms[start_ms : end_ms])
        data['domega_500ms'].extend(omega_500ms[start_ms : end_ms])
        data['domega_1000ms'].extend(omega_1000ms[start_ms : end_ms])

        # get firing rate
        for i, unit in units.iterrows():
            data[unit.unit_id].extend(unit.firing_rate_ms[start_ms : end_ms])

        # get curvature
        for k_cm in curv_sample_points:
            for s in S:
                idx = np.argmin((track_data.S - s)**2)
                data[f"curv_{k_cm}cm"].append(track_data[f"k_{k_cm}"][idx])


        # ensure all entries have the same number of samples
        lengths = set([len(v) for v in data.values()])
        if len(lengths) > 1:
            lns = {k:len(v) for k,v in data.items()}        
            raise ValueError(f"Lengths of data are not the same:\n{lns}")

    
        pd.DataFrame(data).to_hdf(bout_savepath, key="data")
        del data

    del units
    del body
    del bouts

    

